<a href="https://colab.research.google.com/github/juliatessler/1s2023-unicamp-dl-for-search-systems/blob/main/10-trade-offs/10_tradeoffs_pyterrier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9- Efficiency vs Quality trade-offs
by Júlia Tessler

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_doc2query.git -q
!pip install python-terrier -q
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git -q
!pip install faiss-cpu -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 54.2 MB/s eta 0:00:00


## doct5query
[Code reference](https://huggingface.co/spaces/terrierteam/doc2query)

In [ ]:
import pyterrier as pt
pt.init()
import pyterrier_doc2query
import pyterrier_colbert

terrier-assemblies 5.7 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.7 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7



In [ ]:
doc2query = pyterrier_doc2query.Doc2Query(append = True)

dataset = pt.get_dataset('irds:beir/trec-covid')

indexer = pt.IterDictIndexer('./trec-covid')

indxer_pipe = doc2query >> indexer
doc2query_index = indxer_pipe.index(dataset.get_corpus_iter())

[INFO] [starting] building docstore
[INFO] [starting] opening zip file
[INFO] If you have a local copy of https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/ce62140cb23feb9becf6270d0d1fe6d1
[INFO] [starting] https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip
docs_iter:   0%|                                    | 0/171332 [00:01<?, ?doc/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip: 0.0%| 0.00/73.9M [00:00<?, ?B/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip: 0.1%| 49.2k/73.9M [00:00<03:59, 308kB/s]
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip: 0.2%| 115k/73.9M [00:00<03:56, 312kB/s] 
https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip: 0.4%| 262k/73.9M [00:00<02:39, 462kB/s]
https://public.ukp.informa

beir/trec-covid documents:   0%|          | 0/171332 [00:00<?, ?it/s]

In [ ]:
doc2query_br = pt.BatchRetrieve(doc2query_index, wmodel="BM25")

## ColBERT


In [ ]:
!wget https://huggingface.co/castorini/tct_colbert-msmarco/blob/main/pytorch_model.bin

--2023-05-10 17:18:26--  https://huggingface.co/castorini/tct_colbert-msmarco/blob/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 18.155.68.38, 18.155.68.116, 18.155.68.121, ...
Connecting to huggingface.co (huggingface.co)|18.155.68.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52081 (51K) [text/html]
Saving to: ‘pytorch_model.bin’

pytorch_model.bin   100%[===================>]  50.86K  --.-KB/s    in 0.006s  

2023-05-10 17:18:27 (8.55 MB/s) - ‘pytorch_model.bin’ saved [52081/52081]



In [ ]:
import pyterrier_colbert.indexing


In [ ]:
checkpoint="http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip"

In [ ]:
indexer = pyterrier_colbert.indexing.ColBERTIndexer(checkpoint, "/content", "colbertindex", chunksize=3)

In [ ]:
indexer.index(pt.get_dataset("irds:beir/trec-covid").get_corpus_iter())

beir/trec-covid documents:   0%|          | 0/171332 [00:00<?, ?it/s]

[May 10, 17:25:27] [0] 		 #> Local args.bsize = 128
[May 10, 17:25:27] [0] 		 #> args.index_root = /content
[May 10, 17:25:27] [0] 		 #> self.possible_subset_sizes = [69905]


Some weights of the model checkpoint at bert-base-uncased were not used when initializing ColBERT: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ColBERT from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ColBERT were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['linear.weight']
You should probably TRAI

[May 10, 17:25:30] #> Loading model checkpoint.
[May 10, 17:25:30] #> Loading checkpoint https://huggingface.co/castorini/tct_colbert-msmarco/blob/main/pytorch_model.bin


UnpicklingError: ignored

In [ ]:
pyterrier_colbert_factory = indexer.ranking_factory()

colbert_e2e = pyterrier_colbert_factory.end_to_end()

In [ ]:
pt.Experiment(
    [doc2query_br],# colbert_e2e],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=["map", "recip_rank", 'ndcg_cut_5', 'ndcg_cut_10', 'mrt'],
    names=["doc2query"],# "ColBERT"]
)

There are multiple query fields available: ('text', 'query', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,map,recip_rank,ndcg_cut_5,ndcg_cut_10,mrt
0,doc2query,0.180289,0.769227,0.553783,0.541783,54.909733
